<a href="https://colab.research.google.com/github/Observersss/numeral-methods/blob/main/lab_2/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# import numpy as np

# # 1. Генерація матриці 4x4 і вектора правої частини
# def generate_matrix_and_vector(size=4, modulus=10):
#     matrix = np.random.randint(-modulus + 1, modulus, size=(size, size))
#     vector = np.random.randint(-modulus + 1, modulus, size=size)
#     return matrix, vector

import numpy as np

BORDER_LENGTH = 100

def create_matrix_and_vector(n):
    A = np.eye(n)
    b = np.zeros(n)

    for k in range(n):
        A[k, k] = 1 + k * 0.1
    for k in range(1, n):
        A[k, 0] = 1
    for k in range(1, n - 1):
        A[k + 1, k] = 1
    A[0, n - 1] = 1

    for k in range(0, n - 1):
        b[k] = np.sin(np.pi / (n - k))

    return A, b

def check_gauss(matr):
    print("\nChecking if this system of linear computations can be solved with Gauss method:")

    det = np.linalg.det(matr)
    print("Determinant: ", det)
    if(det == 0):
        print("Determinant is 0 -> cannot be solved")
        print("-" * BORDER_LENGTH)
        return False

    print("All good")
    print("-" * BORDER_LENGTH)

    return True

def gauss(A, b):
    print("GAUSS METHOD")

    if(check_gauss(A) == False):
        return np.zeros(len(b))

    matr_len = len(b)
    for i in range(matr_len):
        p = np.identity(matr_len)
        m = np.identity(matr_len)

        print("Current A on step", i, ":\n", A)
        print("\nCurrent b on step", i, ":\n", b)

        max_row = np.argmax(np.abs(A[i:matr_len, i])) + i
        print("\nMax Row Ind:", max_row)

        p[[i, max_row]] = p[[max_row, i]]
        print("\nP matrix:\n", p)

        A = np.matmul(p, A)
        b = np.matmul(p, b.transpose())

        for j in range(matr_len):
            if(j < i):
                m[j, i] = 0
            elif(j == i):
                m[j, i] = 1 / A[j, i]
            else :
                m[j, i] = -A[j, i] / A[i, i]

        print("\nM matrix:\n", m)
        print("-" * BORDER_LENGTH)

        A = np.matmul(m, A)
        b = np.matmul(m, b.transpose())

    print("Result A:\n", A)
    print("\nResult b:", b)
    print("-" * BORDER_LENGTH)

    x = np.zeros(matr_len)
    for i in range(matr_len - 1, -1, -1):
        x[i] = b[i]
        for j in range(i + 1, matr_len):
            x[i] -= A[i, j] * x[j]

    return x

# 4. Обернена матриця через метод Гаусса
def inverse_matrix(A):
    n = len(A)
    augmented_matrix = np.hstack((A, np.identity(n)))
    for i in range(n):
        augmented_matrix[i] = augmented_matrix[i] / augmented_matrix[i][i]
        for j in range(n):
            if i != j:
                augmented_matrix[j] = augmented_matrix[j] - augmented_matrix[j][i] * augmented_matrix[i]
    return augmented_matrix[:, n:]

# 5. Метод Якобі (ітераційний метод)
def jacobi_method(A, b, epsilon=1e-5, max_iterations=100):
    n = len(b)
    x = np.zeros_like(b)
    for it_count in range(max_iterations):
        x_new = np.zeros_like(x)
        for i in range(n):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (b[i] - s1 - s2) / A[i, i]
        if np.linalg.norm(x_new - x, ord=np.inf) < epsilon:
            print(f"Якобі метод збігся за {it_count + 1} ітерацій.")
            return x_new
        x = x_new
    print("Якобі метод не збігся за максимальну кількість ітерацій.")
    return x

# 6. Метод прогонки (для тридіагональних матриць)
def tridiagonal_solve(a, b, c, d):
    n = len(d)
    alpha = np.zeros(n - 1)
    beta = np.zeros(n)
    x = np.zeros(n)

    alpha[0] = -c[0] / b[0]
    beta[0] = d[0] / b[0]

    for i in range(1, n - 1):
        alpha[i] = -c[i] / (a[i - 1] * alpha[i - 1] + b[i])
    for i in range(1, n):
        beta[i] = (d[i] - a[i - 1] * beta[i - 1]) / (a[i - 1] * alpha[i - 1] + b[i])

    x[-1] = beta[-1]
    for i in range(n - 2, -1, -1):
        x[i] = alpha[i] * x[i + 1] + beta[i]

    return x

# Основна функція для запуску всіх методів
def main():
    # Генерація матриці та вектора
    A, b = create_matrix_and_vector(4)
    print("Матриця A:")
    print(A)
    print("Вектор b:")
    print(b)

    # Метод Гаусса
    A_gauss = A.copy()
    b_gauss = b.copy()
    solution_gauss = gauss(A_gauss, b_gauss)
    print("\nРозв'язок методом Гаусса:")
    print(solution_gauss)

    # # Обчислення визначника
    # A_det = A.copy()
    # determinant = calculate_determinant(A_det)
    # print("\nВизначник матриці A:")
    # print(determinant)

    # Обернена матриця
    A_inv = A.copy()
    inv_matrix = inverse_matrix(A_inv)
    print("\nОбернена матриця A:")
    print(inv_matrix)

    # Метод Якобі
    epsilon = float(input("\nВведіть бажану точність для методу Якобі: "))
    solution_jacobi = jacobi_method(A, b, epsilon)
    print("\nРозв'язок методом Якобі:")
    print(solution_jacobi)

    # Перевірка для методу прогонки
    if np.allclose(A, np.diag(np.diag(A))):
        print("\nМатриця є тридіагональною. Використовуємо метод прогонки.")
        # Тут потрібно отримати тридіагональні компоненти a, b, c
        a = np.diag(A, -1)
        b = np.diag(A)
        c = np.diag(A, 1)
        solution_tridiagonal = tridiagonal_solve(a, b, c, b)
        print("\nРозв'язок методом прогонки:")
        print(solution_tridiagonal)
    else:
        print("\nМатриця не є тридіагональною. Метод прогонки не застосовується.")

if __name__ == "__main__":
    main()


Матриця A:
[[1.  0.  0.  1. ]
 [1.  1.1 0.  0. ]
 [1.  1.  1.2 0. ]
 [1.  0.  1.  1.3]]
Вектор b:
[0.70710678 0.8660254  1.         0.        ]
GAUSS METHOD

Checking if this system of linear computations can be solved with Gauss method:
Determinant:  0.49600000000000016
All good
----------------------------------------------------------------------------------------------------
Current A on step 0 :
 [[1.  0.  0.  1. ]
 [1.  1.1 0.  0. ]
 [1.  1.  1.2 0. ]
 [1.  0.  1.  1.3]]

Current b on step 0 :
 [0.70710678 0.8660254  1.         0.        ]

Max Row Ind: 0

P matrix:
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]

M matrix:
 [[ 1.  0.  0.  0.]
 [-1.  1.  0.  0.]
 [-1.  0.  1.  0.]
 [-1.  0.  0.  1.]]
----------------------------------------------------------------------------------------------------
Current A on step 1 :
 [[ 1.   0.   0.   1. ]
 [ 0.   1.1  0.  -1. ]
 [ 0.   1.   1.2 -1. ]
 [ 0.   0.   1.   0.3]]

Current b on step 1 :
 [ 0.70710678  0.15891862  0.2